Reference: https://www.kaggle.com/alvations/word2vec-embedding-using-gensim-and-nltk

In [2]:
cd ..

/Users/nguyen/projects/archi


In [61]:
import re
import gensim
from nltk.corpus import brown

In [51]:
%time model = gensim.models.Word2Vec(brown.sents())

CPU times: user 21.8 s, sys: 1.5 s, total: 23.3 s
Wall time: 18.4 s


In order to persist the model, the model must be saved

In [52]:
model.save('src/models/brown.embedding')

In [54]:
new_model = gensim.models.Word2Vec.load('src/models/brown.embedding')

In [56]:
len(new_model['university'])


/Users/nguyen/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


100

In [57]:
new_model.similarity('university','school')

/Users/nguyen/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  """Entry point for launching an IPython kernel.


0.84171617620747341

Before training a model with IBC chapter 16, the text file needs to be transformed to the same format as brown.sents(), list of lists.

In [58]:
brown.sents()

[['The', 'Fulton', 'County', 'Grand', 'Jury', 'said', 'Friday', 'an', 'investigation', 'of', "Atlanta's", 'recent', 'primary', 'election', 'produced', '``', 'no', 'evidence', "''", 'that', 'any', 'irregularities', 'took', 'place', '.'], ['The', 'jury', 'further', 'said', 'in', 'term-end', 'presentments', 'that', 'the', 'City', 'Executive', 'Committee', ',', 'which', 'had', 'over-all', 'charge', 'of', 'the', 'election', ',', '``', 'deserves', 'the', 'praise', 'and', 'thanks', 'of', 'the', 'City', 'of', 'Atlanta', "''", 'for', 'the', 'manner', 'in', 'which', 'the', 'election', 'was', 'conducted', '.'], ...]

In [5]:
with open('data/corpus/wa-ibc-ch16.txt', 'r') as f:
    corpus = f.read()

In [60]:
corpus

'The provisions of this chapter shall govern the structural design of buildings, structures and portions thereof regulated by this code.The following terms are defined in Chapter 2:   ALLOWABLE STRESS DESIGN.   DEAD LOADS.   DESIGN STRENGTH.   DIAPHRAGM.   Diaphragm, blocked.   Diaphragm boundary.   Diaphragm chord.   ESSENTIAL FACILITIES.   FABRIC PARTITION.   FACTORED LOAD.   HELIPAD.   ICE-SENSITIVE STRUCTURE.   IMPACT LOAD.   LIMIT STATE.   LIVE LOAD.   LIVE LOAD (ROOF).   LOAD AND RESISTANCE FACTOR DESIGN (LRFD).   LOAD EFFECTS.   LOAD FACTOR.   LOADS.   NOMINAL LOADS.   OTHER STRUCTURES.   PANEL (PART OF A STRUCTURE).   RESISTANCE FACTOR.   RISK CATEGORY.   STRENGTH, NOMINAL.   STRENGTH, REQUIRED.   STRENGTH DESIGN.   SUSCEPTIBLE BAY.   VEHICLE BARRIER.   NOTATIONS.    D = Dead load.  Di = Weight of ice in accordance with Chapter 10 of ASCE 7.  E = Combined effect of horizontal and vertical earthquake induced forces as defined in Section 12.4.2 of ASCE 7.  F = Load due to fluids 

In [65]:
caps = "([A-Z])"
prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov)"

def split_into_sentences(text):
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + caps + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(caps + "[.]" + caps + "[.]" + caps + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(caps + "[.]" + caps + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + caps + "[.]"," \\1<prd>",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(". ",".<stop> ")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    return sentences

In [91]:
sentences = split_into_sentences(corpus)
len(sentences)

sentences = [sent for sent in sentences if len(sent) > 3]
len(sentences)

600

In [92]:
# remove periods at the end of the sentence
for idx, sent in enumerate(sentences):
    if sent[-1] == '.':
        sentences[idx] = sent[0:-1]

In [93]:
# remove ':  1' at the end of sentences
for idx, sent in enumerate(sentences):
    if sent[-5:] == ':   1':
        sentences[idx] = sent[0:-4]

In [90]:
# delete Tables
# for idx, sent in enumerate(sentences):
#     if sent.split()[0] == 'TABLE':
#         del sentences[idx]

In [96]:
sentences = [sent.split() for sent in sentences]

In [97]:
sentences

[['The',
  'provisions',
  'of',
  'this',
  'chapter',
  'shall',
  'govern',
  'the',
  'structural',
  'design',
  'of',
  'buildings,',
  'structures',
  'and',
  'portions',
  'thereof',
  'regulated',
  'by',
  'this',
  'code.The',
  'following',
  'terms',
  'are',
  'defined',
  'in',
  'Chapter',
  '2:',
  'ALLOWABLE',
  'STRESS',
  'DESIGN'],
 ['DEAD', 'LOADS'],
 ['DESIGN', 'STRENGTH'],
 ['DIAPHRAGM'],
 ['Diaphragm,', 'blocked'],
 ['Diaphragm', 'boundary'],
 ['Diaphragm', 'chord'],
 ['ESSENTIAL', 'FACILITIES'],
 ['FABRIC', 'PARTITION'],
 ['FACTORED', 'LOAD'],
 ['HELIPAD'],
 ['ICE-SENSITIVE', 'STRUCTURE'],
 ['IMPACT', 'LOAD'],
 ['LIMIT', 'STATE'],
 ['LIVE', 'LOAD'],
 ['LIVE', 'LOAD', '(ROOF)'],
 ['LOAD', 'AND', 'RESISTANCE', 'FACTOR', 'DESIGN', '(LRFD)'],
 ['LOAD', 'EFFECTS'],
 ['LOAD', 'FACTOR'],
 ['LOADS'],
 ['NOMINAL', 'LOADS'],
 ['OTHER', 'STRUCTURES'],
 ['PANEL', '(PART', 'OF', 'A', 'STRUCTURE)'],
 ['RESISTANCE', 'FACTOR'],
 ['RISK', 'CATEGORY'],
 ['STRENGTH,', 'NOMINAL'

In [98]:
%time ch16_model = gensim.models.Word2Vec(sentences)

CPU times: user 82.8 ms, sys: 8.91 ms, total: 91.8 ms
Wall time: 78.4 ms


In [99]:
ch16_model.save('src/models/ibc_ch16.embedding')

In [101]:
new_model = gensim.models.Word2Vec.load('src/models/ibc_ch16.embedding')

In [114]:
new_model.similarity('where', 'wind')

/Users/nguyen/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  """Entry point for launching an IPython kernel.


0.99982539979178353

In [ ]:
new_mode